In [1]:
import pyspark.pandas as ps

/home/jag/.conda/envs/newest_model_engine/lib/python3.10/site-packages/pyspark/pandas/__init__.py:43: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [82]:
import pandas as pd
pd.set_option('display.float_format', '{:.6f}'.format)
import pickle
import json


In [9]:
def load_obj(folder, substring):
    location = f'{folder}{substring}'
    with open(location, 'rb') as f:
        meta_data = pickle.load(f)
        pipe = pickle.load(f)
    return pipe

In [10]:
path_final_CO36best_egg_pretest_folded_in =  '/d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/exp7.6.CO36m.pre_test.all_features_foldinvalid/'
path_final_CO36best_egg_pretest_no_pretest_folded_in = '/d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/exp7.5.CO36m.pre_test.all_features_foldinvalid/'
path_final_LGD_best_egg =  '/d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/lgd_model/exp2/'

In [11]:
pipe_co36_best_egg = load_obj(folder =path_final_CO36best_egg_pretest_no_pretest_folded_in, substring = 'pipeline.obj')
model_lgd_bestegg = load_obj(folder =path_final_LGD_best_egg, substring = 'model.obj')
calibration_object_co36_best_egg = load_obj(folder = path_final_CO36best_egg_pretest_no_pretest_folded_in, substring = 'calibration_object.obj')

/home/jag/.conda/envs/newest_model_engine/lib/python3.10/site-packages/zaml/common/utils/io.py:17: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources



In [17]:
feature_names_co_36 = pipe_co36_best_egg.model.model.feature_names
feature_names_lgd = model_lgd_bestegg.model.feature_names

print(f'''Number of Features in CO 36 Model: {len(feature_names_co_36)}''')

print(f'''Number of Features in LGD Model: {len(feature_names_lgd)}''')

Number of Features in CO 36 Model: 734
Number of Features in LGD Model: 53


0.1961472046715804

# We are following  [this notebook](https://github.com/Katlean/client-project-bestegg/blob/master/personalloan/modeling/all_features/LGD_v3/generate_lgd_evaluation_metrics_36m.ipynb) but with spark

In [88]:
data_split =  {
            "train": {
                "start_date": "2021-12-01",
                "end_date": "2024-02-28"
            },
            "valid":{
                "start_date": "2024-03-01",
                "end_date":"2024-03-31"
            },
            "test": {
                "start_date": "2024-04-01",
                "end_date":"2024-06-30"
            },
            "pre_test": {
                "start_date": "2021-10-01",
                "end_date":"2021-11-30"
            }
}

eval_output_folder = '/d/shared/silver_projects_v2/bestegg/personalloanv1/shared_data/experiments/lgd_results_all_features_v3/metrics'
pred_location = "/d/shared/silver_projects_v2/bestegg/personalloanv1/shared_data/experiments/lgd_results_all_features_v3"
pre_test_all_unit_path = '/d/shared/silver_projects_v2/bestegg/personalloanv1/shared_data/processed_data/all_features/lgd_modeling_dataset_v2/lgd_all_data_ageatdefault_v2.parquet'
pre_test_all_data_path = '/d/shared/silver_projects_v2/bestegg/personalloanv1/shared_data/processed_data/all_features/PL_DOLLAR_LOSS_PERF_2021Q4_20251210_hit_processed.parquet'

# Prepare Pre_test_all_unit

In [89]:
import numpy as np

# PReparing PRe_Test_All_unit

In [90]:
pre_test_all_unit = pd.read_parquet(pre_test_all_unit_path, columns = ['unit_target'])

## we fill na with -1

In [91]:
pre_test_all_unit.unit_target = pre_test_all_unit.unit_target.fillna(-1)

# Preparing pre_test_all_data

In [92]:
pre_test_all_data = pd.read_parquet(pre_test_all_data_path, columns = ['APP_DATE', 'TERMS_DURATION', 'FUNDED_FLAG', 'ORIGINAL_TERM', 'TERMS_FREQUENCY', 'OFFERED'])

## Subset app date to being greater than the pre test start date and before the pretest end date

In [93]:
pre_test_all_data['APP_DATE'] = pd.to_datetime(pre_test_all_data['APP_DATE'])

pre_test_all_data = pre_test_all_data[(pre_test_all_data['APP_DATE'] >= data_split['pre_test']['start_date']) & (pre_test_all_data['APP_DATE'] <= data_split['pre_test']['end_date'])]

## Terms Duration that are UNNK Rev, LOC We make it NAN

In [94]:
pre_test_all_data['TERMS_DURATION'] = (
pre_test_all_data['TERMS_DURATION']
.replace(['UNK', 'REV', 'LOC'], np.nan)
.astype(float)
)

# Proxy Term is the terms duration

In [95]:
pre_test_all_data['proxy_term'] = pre_test_all_data['TERMS_DURATION']

### convert the proxy term into number of months

### For examle if weekly make it divide by 4 to get in number of months


In [96]:
pre_test_all_data.loc[pre_test_all_data['TERMS_FREQUENCY'].isin(['B', 'E']), 'proxy_term'] = pre_test_all_data['TERMS_DURATION'] / 2
pre_test_all_data.loc[pre_test_all_data['TERMS_FREQUENCY'] == 'W', 'proxy_term'] = pre_test_all_data['TERMS_DURATION'] / 4
pre_test_all_data.loc[pre_test_all_data['TERMS_FREQUENCY'] == 'Q', 'proxy_term'] = pre_test_all_data['TERMS_DURATION'] * 3
pre_test_all_data.loc[pre_test_all_data['TERMS_FREQUENCY'] == 'S', 'proxy_term'] = pre_test_all_data['TERMS_DURATION'] * 6
pre_test_all_data.loc[pre_test_all_data['TERMS_FREQUENCY'] == 'Y', 'proxy_term'] = pre_test_all_data['TERMS_DURATION'] * 12

In [97]:
original_term_is_na = pre_test_all_data['ORIGINAL_TERM'].isna()

In [98]:
np.sum(original_term_is_na)/len(pre_test_all_data)

0.8754520790476125

## 85 percent of our data was a proxy term. Is that what this part is saying?

In [99]:
pre_test_all_data['all_terms'] = pre_test_all_data['ORIGINAL_TERM'].fillna(pre_test_all_data['proxy_term']).astype(float)

## Subset only to people whose term was between 36 and 24 (will all be 36 for people with non-nan original)

In [100]:
pre_test_all_data['all_terms'] = pre_test_all_data['all_terms'].astype(float)


## We can see that people who have an original term  are almost all 36 or 60. Makes sense

In [101]:
pre_test_all_data[original_term_is_na]['all_terms'].describe()

count   380848.000000
mean        41.426192
std         29.299872
min          0.250000
25%         24.000000
50%         36.000000
75%         60.000000
max       1008.000000
Name: all_terms, dtype: float64

In [102]:
pre_test_all_data[~original_term_is_na]['all_terms'].value_counts()

all_terms
36.000000    29415
60.000000    21180
24.000000     2031
48.000000     1682
Name: count, dtype: int64

In [103]:
pre_test_all_data[~original_term_is_na]['all_terms'].describe()

count   54308.000000
mean       45.282831
std        12.178493
min        24.000000
25%        36.000000
50%        36.000000
75%        60.000000
max        60.000000
Name: all_terms, dtype: float64

In [104]:
pre_test_all_data[~original_term_is_na][['OFFERED', 'all_terms']].value_counts()

OFFERED  all_terms
1.0      36.000000    29415
         60.000000    21180
         24.000000     2031
         48.000000     1682
Name: count, dtype: int64

In [105]:
pre_test_eval = pre_test_all_data[(pre_test_all_data['all_terms'] >= 24) & (pre_test_all_data['all_terms'] <= 36)] #change point

## We then create funded flag as a numeric and ones that don't match are NAN
## same with offered

In [106]:
pre_test_eval["FUNDED_FLAG"] = pd.to_numeric(pre_test_eval["FUNDED_FLAG"], errors="coerce")
pre_test_eval["OFFERED"] = pd.to_numeric(pre_test_eval["OFFERED"], errors="coerce")
pre_test_eval['FUNDED_FLAG'].value_counts()

FUNDED_FLAG
0.000000    102989
1.000000     31446
Name: count, dtype: int64

## First we merge the funding information and the terms information with the target and then we merge the unit target to that

In [107]:
target_pre_test = pd.read_parquet('/d/shared/silver_projects_v2/bestegg/personalloanv1/shared_data/processed_data/all_features/CO_targets_proxy_v2/PL_DOLLAR_LOSS_PERF_2021Q4_20251210_hit_processed_proxy_target.parquet')

In [108]:
pre_test_all_data_target = pre_test_eval.merge(target_pre_test['CO_final_target'], left_index=True, right_index=True, how='left')\
                                    .merge(pre_test_all_unit['unit_target'], left_index=True, right_index=True, how='left')

### Note that pre_test_all_unit seems to have all the data

In [109]:
pre_test_all_data_target.shape, pre_test_eval.shape, pre_test_all_unit.shape

((134435, 10), (134435, 8), (1130026, 1))

In [110]:
mask_charged_off = pre_test_all_data_target['CO_final_target']==1.0

In [111]:
pre_test_all_data_target[mask_charged_off]['unit_target'].describe()

count   30477.000000
mean             inf
std              NaN
min        -1.000000
25%         0.341400
50%         0.675269
75%         0.888216
max              inf
Name: unit_target, dtype: float64

In [112]:
pre_test_all_data_target[~mask_charged_off]['unit_target'].describe()

count   124.000000
mean     -1.000000
std       0.000000
min      -1.000000
25%      -1.000000
50%      -1.000000
75%      -1.000000
max      -1.000000
Name: unit_target, dtype: float64

## We subet to only have people who did charge off

In [121]:
pre_test_all_data_target_no_nan_unit_target = pre_test_all_data_target[~(pre_test_all_data_target.unit_target==-1)]


In [122]:
pre_test_all_data_target_no_nan_unit_target.unit_target.describe(percentiles = (.1,.2,.3,.4,.5,.6,.7,.8,.9,.96))

count   26219.000000
mean             inf
std              NaN
min         0.000133
10%         0.295579
20%         0.439355
30%         0.553438
40%         0.656886
50%         0.747273
60%         0.821200
70%         0.883900
80%         0.940867
90%         1.000000
96%         1.093333
max              inf
Name: unit_target, dtype: float64

In [120]:
pre_test_all_data_target_no_nan_unit_target.unit_target = pre_test_all_data_target_no_nan_unit_target.unit_target.fillna(0)
pre_test_all_data_target_no_nan_unit_target.unit_target.describe(percentiles = (.1,.2,.3,.4,.5,.6,.7,.8,.9,.96))

count   130053.000000
mean              inf
std               NaN
min          0.000000
10%          0.000000
20%          0.000000
30%          0.000000
40%          0.000000
50%          0.000000
60%          0.000000
70%          0.000000
80%          0.043825
90%          0.750557
96%          0.941811
max               inf
Name: unit_target, dtype: float64

In [ ]:
pre_test_all_data_target_no_nan_unit_target

In [114]:
pre_test_all_data_target['CO_final_target'].value_counts()

CO_final_target
0.000000    103958
1.000000     30477
Name: count, dtype: int64

In [118]:
pre_test_all_data_target[pre_test_all_data_target['CO_final_target']==1]['unit_target'].describe(percentiles = (.1,.2,.3,.4,.5,.6,.7,.8,.9,.96))

count   30477.000000
mean             inf
std              NaN
min        -1.000000
10%        -1.000000
20%         0.241415
30%         0.421512
40%         0.556048
50%         0.675269
60%         0.773309
70%         0.854213
80%         0.922055
90%         0.984423
96%         1.076196
max              inf
Name: unit_target, dtype: float64

In [119]:
pre_test_all_data_target[pre_test_all_data_target['CO_final_target']==0]['unit_target'].describe(percentiles = (.1,.2,.3,.4,.5,.6,.7,.8,.9,.96))

count   124.000000
mean     -1.000000
std       0.000000
min      -1.000000
10%      -1.000000
20%      -1.000000
30%      -1.000000
40%      -1.000000
50%      -1.000000
60%      -1.000000
70%      -1.000000
80%      -1.000000
90%      -1.000000
96%      -1.000000
max      -1.000000
Name: unit_target, dtype: float64

# understnad


In [123]:
pred_loc = ['lgd_36m_exp1_lessthan_2','lgd_36m_exp4_all_vals_nocap',
           'lgd_36m_exp7_co_ever_preds_feature']

In [133]:
pred_df_lgd_36m_exp_less_than_1 = pd.read_parquet(pred_location+'/'+pred_loc[0]+'/pre_test_preds.parquet').reset_index()

In [126]:
pred_df_lgd_36m_exp_less_than_1['lgd_prediction'].describe()

count   436041.000000
mean         0.672485
std          0.109524
min          0.349844
25%          0.590236
50%          0.669929
75%          0.752596
max          1.011066
Name: lgd_prediction, dtype: float64

In [127]:
test = pd.read_parquet(pre_test_all_data_path, columns = ['APP_DATE', 'TERMS_DURATION', 'FUNDED_FLAG', 'ORIGINAL_TERM', 'TERMS_FREQUENCY', 'OFFERED'])

In [130]:
len(target_pre_test)/len(pred_df_lgd_36m_exp_less_than_1)

1.5037462073520609

In [132]:
test_target_pretest = target_pre_test.reset_index()[['ZEST_KEY', 'CO_final_target']]

In [134]:
co_target_and_pred = test_target_pretest.merge(pred_df_lgd_36m_exp_less_than_1, on = ['ZEST_KEY'], how = 'left')

In [136]:
lgd_nan = co_target_and_pred['lgd_prediction'].isna()



In [137]:
co_target_and_pred[lgd_nan]['CO_final_target'].value_counts()

CO_final_target
0.000000    155891
1.000000     63763
Name: count, dtype: int64

In [138]:
co_target_and_pred[~lgd_nan]['CO_final_target'].value_counts()

CO_final_target
0.000000    313713
1.000000    122327
Name: count, dtype: int64